In [34]:
!wget http://cs.stanford.edu/people/alecmgo/trainingandtestdata.zip

URL transformed to HTTPS due to an HSTS policy
--2020-04-21 05:48:27--  https://cs.stanford.edu/people/alecmgo/trainingandtestdata.zip
Resolving cs.stanford.edu (cs.stanford.edu)... 171.64.64.64
Connecting to cs.stanford.edu (cs.stanford.edu)|171.64.64.64|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 81363704 (78M) [application/zip]
Saving to: ‘trainingandtestdata.zip.1’

trainingandtestdata 100%[===================>]  77.59M  20.7MB/s    in 4.7s    

2020-04-21 05:48:32 (16.6 MB/s) - ‘trainingandtestdata.zip.1’ saved [81363704/81363704]



In [0]:
import zipfile
data_ref = "./trainingandtestdata.zip"

with zipfile.ZipFile(data_ref, 'r') as f:
  f.extractall()

In [0]:
training_file = "training.1600000.processed.noemoticon.csv"
testing_file = "testdata.manual.2009.06.14.csv"
import numpy as np
import pandas as pd

In [0]:
train_df = pd.read_csv(training_file, encoding = "ISO-8859-1", names=["polarity", "id", "date", "query", "user", "text"])

In [38]:
test_df = pd.read_csv(testing_file, encoding = "ISO-8859-1", names=["polarity", "id", "date", "query", "user", "text"])
test_df.head()

,polarity,id,date,query,user,text
0,4,3,Mon May 11 03:17:40 UTC 2009,kindle2,tpryan,@stellargirl I loooooooovvvvvveee my Kindle2. ...
1,4,4,Mon May 11 03:18:03 UTC 2009,kindle2,vcu451,Reading my kindle2... Love it... Lee childs i...
2,4,5,Mon May 11 03:18:54 UTC 2009,kindle2,chadfu,"Ok, first assesment of the #kindle2 ...it fuck..."
3,4,6,Mon May 11 03:19:04 UTC 2009,kindle2,SIX15,@kenburbary You'll love your Kindle2. I've had...
4,4,7,Mon May 11 03:21:41 UTC 2009,kindle2,yamarama,@mikefish Fair enough. But i have the Kindle2...


In [39]:
train_df["text"][0]

"@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer.  You shoulda got David Carr of Third Day to do it. ;D"

In [40]:
import string
import re
import nltk
nltk.download('all')
from nltk import word_tokenize

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Package abc is already up-to-date!
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Package alpino is already up-to-date!
[nltk_data]    | Downloading package biocreative_ppi to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package biocreative_ppi is already up-to-date!
[nltk_data]    | Downloading package brown to /root/nltk_data...
[nltk_data]    |   Package brown is already up-to-date!
[nltk_data]    | Downloading package brown_tei to /root/nltk_data...
[nltk_data]    |   Package brown_tei is already up-to-date!
[nltk_data]    | Downloading package cess_cat to /root/nltk_data...
[nltk_data]    |   Package cess_cat is already up-to-date!
[nltk_data]    | Downloading package cess_esp to /root/nltk_data...
[nltk_data]    |   Package cess_esp is already up-to-date!
[nltk_data]    | Downloading packag

In [0]:
texts = train_df['text'].to_numpy()
labels = train_df['polarity'].to_numpy()
test_texts = test_df['text'].to_numpy()
test_labels = test_df['polarity'].to_numpy()
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
from nltk.corpus import stopwords 
stop_words = set(stopwords.words('english'))

In [0]:
def clean_text(text):
  tokens = text.split(' ');
  tokens = [token.lower() for token in tokens if token not in stop_words]
  tokens = [lemmatizer.lemmatize(token) for token in tokens if not re.search('^@', token) and not re.search('^http', token) and not re.search('^\:', token) and not re.search('^\;', token)]
  new_text = ' '.join(tokens)
  return ''.join(ch for ch in new_text if ch not in string.punctuation)

In [0]:
def process_text(texts):
  cleaned_texts = []
  for text in texts:
    cleaned_texts.append(clean_text(text))
  cleaned_texts = np.array(cleaned_texts)
  return cleaned_texts

In [0]:
cleaned_texts = process_text(texts)

In [0]:
cleaned_test_texts = process_text(test_texts)

In [0]:
import tensorflow as tf
from tensorflow.keras.layers import LSTM, GRU, Flatten, Dense, Embedding, Bidirectional, GlobalAveragePooling1D
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential

In [0]:
vocab_size = 1000
max_length = 100
embedding_dim = 16
trunc_type = "post"
pad_type = "post"
oov_token = '<OOV>'

In [0]:
tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_token)
tokenizer.fit_on_texts(cleaned_texts)
word_index = tokenizer.word_index
train_sequences = tokenizer.texts_to_sequences(cleaned_texts)
padded_train = pad_sequences(train_sequences, maxlen=max_length,padding=pad_type,truncating=trunc_type)

In [0]:
test_sequences = tokenizer.texts_to_sequences(cleaned_test_texts)
padded_test = pad_sequences(test_sequences, maxlen=max_length,padding=pad_type,truncating=trunc_type)

In [0]:
model = Sequential([
  Embedding(vocab_size, embedding_dim, input_length=max_length),
  Bidirectional(LSTM(32)),
  Dense(6, activation="relu"),
  Dense(1, activation="sigmoid")
])

In [51]:
model.compile(loss="binary_crossentropy", optimizer="adam", metrics=['accuracy'])
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 100, 16)           16000     
_________________________________________________________________
bidirectional_1 (Bidirection (None, 64)                12544     
_________________________________________________________________
dense_2 (Dense)              (None, 6)                 390       
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 7         
Total params: 28,941
Trainable params: 28,941
Non-trainable params: 0
_________________________________________________________________


In [0]:
def process_labels(labels):
  processed_labels = []
  for label in labels:
    if label == 0:
      processed_labels.append(0)
    else:
      processed_labels.append(1)
  return np.array(processed_labels)

In [0]:
labels = process_labels(labels)
test_labels = process_labels(test_labels)

In [0]:
class my_callback(tf.keras.callbacks.Callback):
  def on_epoch_end(self,epoch, logs={}):
    if logs.get('accuracy') > 0.95:
      self.model.stop_training=True


In [57]:
callbacks = my_callback()
nmodel = tf.keras.models.load_model('./sentiment_model.h5')
# nmodel.summary()
history = nmodel.fit(padded_train, labels, epochs=1000, validation_data=(padded_test, test_labels), callbacks=[callbacks])

Epoch 1/1000
50000/50000 [==============================] - 2174s 43ms/step - loss: 0.4767 - accuracy: 0.7646 - val_loss: 0.4617 - val_accuracy: 0.7912
Epoch 2/1000
23870/50000 [=============>................] - ETA: 18:43 - loss: 0.4727 - accuracy: 0.7673

KeyboardInterrupt: ignored

In [0]:
from google.colab import drive
# drive.mount('/content/gdrive');
save_name = "sentiment_model.h5"
path = F"/content/gdrive/My Drive/{save_name}"
nmodel.save(path) 

In [0]:
import pickle

In [0]:
with open('tokenizer.pickle', 'wb') as f:
  pickle.dump(tokenizer, f, protocol=pickle.HIGHEST_PROTOCOL)